# File to work in and test few stuff

In [2]:
import numpy as np
import matplotlib.pyplot as plt


n_nucleo = 10
s_nucleo = 15

n_linker = n_nucleo + 1
s_linker = 5

s_shift = 1

def alpha_diffused(
    n_nucleo: int,
    s_nucleo: int,
    n_linker: int,
    s_linker: int,
    s_shift: int,
    alphaf: float,
    alphao: float
) -> np.ndarray :
    """Generate a 1D alpha profile representing a chromatin fiber composed of nucleosomes and linkers,
    with random diffusion (shifts) of linker regions around nucleosomes.

    Args:
        n_nucleo (int): Number of nucleosomes.
        s_nucleo (int): Size (in base pairs) of each nucleosome.
        n_linker (int): Number of linker regions (typically n_nucleo - 1).
        s_linker (int): Size (in base pairs) of each linker region.
        s_shift (int): Maximum extent (in bp) of the fuzzy linker region on each side of a nucleosome.
                       Must satisfy s_shift ≤ s_linker // 2 to prevent overlapping.
        alphaf (float): Alpha value assigned to fuzzy (linker) regions.
        alphao (float): Alpha value assigned to ordered (nucleosome) regions.

    Returns:
        np.ndarray | None: The resulting 1D alpha profile, or None if the given s_shift is too large.
    
    Notes:
        - The base profile alternates between linker and nucleosome regions with fixed periodicity.
        - Each nucleosome is randomly assigned a side (left or right) where additional linker fuzziness is applied.
        - The final profile is plotted: initial in blue, diffused in red.
    """
    
    if s_shift > s_linker // 2:
        print("No such s_shift value permitted because of possible entanglement")
        return None

    # Random shifts: -1 (right) or 1 (left) for each nucleosome
    # Construct the initial periodic alpha profile
    shifts = np.random.choice([-1, 1], size=n_nucleo, p=[0.5, 0.5])
    s_period = s_linker + s_nucleo
    shifted = np.array(
        (s_linker * [alphaf] + s_nucleo * [alphao]) * n_nucleo + s_linker * [alphaf],
        dtype=float
    )

    # Apply diffusion around each nucleosome
    for i in range(n_nucleo):
        nucleo_start = s_linker + i * s_period
        shifted[nucleo_start - s_shift : nucleo_start + s_nucleo + s_shift] = alphao

        if shifts[i] == 1:
            # Fuzzy region to the left
            shifted[nucleo_start - s_shift : nucleo_start + s_shift] = alphaf
        else:
            # Fuzzy region to the right
            shifted[nucleo_start - s_shift + s_nucleo : nucleo_start + s_shift + s_nucleo] = alphaf

    return shifted


alpha_diffused(n_nucleo, s_nucleo, n_linker, s_linker, s_shift, 1, 0)

array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1.])

In [2]:
import os
os.environ['OMP_NUM_THREADS'] = '1' # prevent multithreading

from tqdm.auto import tqdm
from multiprocessing import Pool

import numpy as np
from scipy import optimize, linalg
from matplotlib import pyplot as plt



# The implementation we are using is not particularly efficient; so let's speed things up by parallelization
N_traj = 10 # the more the better (but longer runtime)
with Pool() as mypool:
    imap = mypool.imap_unordered(run_recoil, np.random.randint(1<<31, size=N_traj))
    imap = tqdm(imap, total=N_traj)
    trajectories = np.array(list(imap))

# os.environ['OMP_NUM_THREADS'] = '1' # prevent multithreading

# from tqdm.auto import tqdm
# from multiprocessing import Pool
# from scipy import optimize, linalg
# from matplotlib import pyplot as plt

plt.rcParams['font.size'] = 8


plt.rcParams['font.size'] = 8    


# ctrl a pour selectionnet toute une cellule d'un coup
# ctrl shift pour passer de fenetre en fenetre
# alt flèches haut bas gauche droite

In [84]:
import numpy as np

x = np.array([[0, 3, 0, 5, 0, 2, 2, 7, 2, 5, 5],
              [0, 5, 5, 7, 7, 9, 7, np.nan, np.nan, np.nan, np.nan],
              [0, 4, 0, 7, 0, 9, 0, 7, 7, np.nan, np.nan]])

t = np.array([[0, 0.22, 0.31, 0.47, 1.20, 1.25, 3.21, 3.56, 4.55, 5.12, 6.12],
              [0, 0.45, 0.55, 0.88, 2.22, 3.12, 3.55, np.nan, np.nan, np.nan, np.nan],
              [0, 0.12, 0.44, 0.76, 0.79, 2.13, 2.55, 4.17, 5.13, np.nan, np.nan]])

print(x, "\n", t)

[[ 0.  3.  0.  5.  0.  2.  2.  7.  2.  5.  5.]
 [ 0.  5.  5.  7.  7.  9.  7. nan nan nan nan]
 [ 0.  4.  0.  7.  0.  9.  0.  7.  7. nan nan]] 
 [[0.   0.22 0.31 0.47 1.2  1.25 3.21 3.56 4.55 5.12 6.12]
 [0.   0.45 0.55 0.88 2.22 3.12 3.55  nan  nan  nan  nan]
 [0.   0.12 0.44 0.76 0.79 2.13 2.55 4.17 5.13  nan  nan]]


In [85]:
# Ce qu'on veut c'est trier 4 choses :
# - Temps d'attente des reverses et des forwards
# - Temps de saut des reverses et des forwards

In [86]:
# Tous les temps d'attente correspondent aux indices paires de la différence
# Tous les temps de saut correspondent aux indices impaires de la différence

diffs_t = np.diff(t, axis=1)
print(diffs_t)

[[0.22 0.09 0.16 0.73 0.05 1.96 0.35 0.99 0.57 1.  ]
 [0.45 0.1  0.33 1.34 0.9  0.43  nan  nan  nan  nan]
 [0.12 0.32 0.32 0.03 1.34 0.42 1.62 0.96  nan  nan]]


In [87]:
dwell_times = diffs_t[:, 0::2]
jump_times = diffs_t[:, 1::2]
print(dwell_times, "\n", jump_times)

[[0.22 0.16 0.05 0.35 0.57]
 [0.45 0.33 0.9   nan  nan]
 [0.12 0.32 1.34 1.62  nan]] 
 [[0.09 0.73 1.96 0.99 1.  ]
 [0.1  1.34 0.43  nan  nan]
 [0.32 0.03 0.42 0.96  nan]]


In [88]:
# Et pour les positions ?
print(x)

# dwell_x = x[:, 0::2]
# jump_x = x[:, 1::2]

# print(dwell_x, "\n", jump_x)

[[ 0.  3.  0.  5.  0.  2.  2.  7.  2.  5.  5.]
 [ 0.  5.  5.  7.  7.  9.  7. nan nan nan nan]
 [ 0.  4.  0.  7.  0.  9.  0.  7.  7. nan nan]]


In [93]:
mask = np.zeros_like(x, dtype=bool)

for i in range(len(x)):
    for j in range (len(x[i])-1):
        if x[i][j]==x[i][j+1]:
            mask[i][j] = True
            mask[i][j+1] = True

print(mask)

[[False False False False False  True  True False False  True  True]
 [False  True  True  True  True False False False False False False]
 [False False False False False False False  True  True False False]]


In [92]:
mask = np.zeros_like(x, dtype=bool)
equal_next = (x[:, :-1] == x[:, 1:])

mask[:, :-1] |= equal_next
mask[:,  1:] |= equal_next

print(mask)

[[False False False False False  True  True False False  True  True]
 [False  True  True  True  True False False False False False False]
 [False False False False False False False  True  True False False]]


# .